In [1]:
#encoding='GB2312'
import sys
import math
import cv2
import numpy as np
import ecal.core.core as ecal_core
#from ecal.core.subscriber import StringSubscriber
from ecal.core.subscriber import ProtoSubscriber
import json
import RadarObject_pb2



scale = 800./600.
edge = int(750 * scale)
max_distance = 270
scale_y = 8.0 * scale
scale_x = math.floor(edge / max_distance)


def deserialize(bytesdata):
    detection_event = RadarObject_pb2.RadarObject()  # 创建一个detection检测事件
    detection_event.ParseFromString(bytesdata)
    print(detection_event.ImageName)
    print(detection_event.timestamp)
    print(detection_event.width)
    print(detection_event.height)
    for target_event in detection_event.TargetList:
        print(target_event.targetId)
        print(target_event.box)
        print(target_event.boxScore)
        print(target_event.labelType)


def angle_diff(a, b):
    d1 = a - b
    d2 = 2 * math.pi - abs(d1)
    if d1 > 0:
        d2 *= -1.0
    if abs(d1) < abs(d2):
        return d1
    else:
        return d2


def wrap_angle_once(angle):
    wrap = angle
    if angle > math.pi:
        wrap -= (2 * math.pi)
    if angle < -math.pi:
        wrap += (2 * math.pi)
    return wrap


def arrow(img, orientation, x, y, len, cl=(0, 0, 0), thickness=1):
    angle = orientation + math.pi / 2.
    alpha = 30.

    _x = x + len * math.cos(angle + math.pi * alpha / 180.)
    _y = y + len * math.sin(angle + math.pi * alpha / 180.)
    _x, _y = int(_x), int(_y)
    cv2.line(img, (x, y), (_x, _y), cl, thickness=thickness)

    _x = x + len * math.cos(angle - math.pi * alpha / 180.)
    _y = y + len * math.sin(angle - math.pi * alpha / 180.)
    _x, _y = int(_x), int(_y)
    cv2.line(img, (x, y), (_x, _y), cl, thickness=thickness)

#转成图像坐标
def translate_x(_x):
    return abs(float((float(_x) - max_distance / 2) * scale_x) - float(edge / 2))


def translate_y(_y):
    return -float(_y) * scale_y + float(edge / 2)

#这个位置是修改线圈位置的，x代表纵向，y代表横向
y_pos = 10
x_pos = 66
w_, l_ = 3.4, 12


def has_enter_coil(x, y):
    return x_pos + l_ > x > x_pos and y_pos > y > y_pos - w_


def main(radar_no):
    print("eCAL {} ({})\n".format(ecal_core.getversion(), ecal_core.getdate()))
    ecal_core.initialize(sys.argv, "cluster_receive")
    ecal_core.set_process_state(1, 1, "I feel good")
    #help(type_protobuf)
    sub408 = ProtoSubscriber("ARS4G0_ObjectListPb_XW%s" % radar_no,RadarObject_pb2.RadarObject)

    enter_coil_ids = set()
    enter_coil_count:int = 0

    azimuth = 0.0
    #雷达相当于车道线的角度值,弧度值,上位机上面调,(上位机是角度,要转成弧度)
    radar_azimuth = {'81': -0.03491, '82': 0.06702, '83': 0.0, '84': 0.0, '215': 0.0}
    radar_stop_line = {'81': [30, 50, 53.1], '82': [28, 50, 53.4], '83': [27, 47, 50.8], '84': [34, 56, 59.35], '215': [20, 30, 39.9]} #第一个是盲区,39.9表示停止线,30表示停止线前移,为了统计效果
    radar_lane_line = {
        '81': [6.27, 3.27, 0.26, -2.72, -5.8, -9.14, -12.48, -15.79, -19.03, -22.33],
        '82': [5.56, 1.96, -1.54, -5.04, -6.4, -9.91, -13.43],
        '83': [9.54, 7.05, 3.7, 0.68, -2.3, -5.61, -9.67, -12.7, -15.66, -19.01],
        '84': [7.54, 3.87, 0.36, -3.15, -4.5, -8, -11.51],
        #从上位机调好,然后录进去
        '215': [9.8,7.1,3.5,0.5,-3,-6.4,-9.8,-13,-15.4]}
    if radar_azimuth.get(radar_no):
        azimuth = radar_azimuth[radar_no]

    #画各种车线
    stop_line = radar_stop_line.get(radar_no)
    lane_line = radar_lane_line.get(radar_no)
    #meas_line = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 200, 250]
    meas_line = [ 20,  60, 80,  100,  120, 140, 160, 200, 240]
    # img_template
    img_tpl = np.ones((edge, edge, 3), np.uint8)
    img_tpl *= 255
    #画雷达安装位置
    #Y 坐标左正右负
    cv2.putText(img_tpl,"Radar",(int(translate_y(-2)),int(translate_x(-1))), cv2.FONT_ITALIC, 0.6, (0, 0, 0), 2)
    cv2.circle(img_tpl, (int(translate_y(0)), int(translate_x(0))), 3, (255, 0, 0), thickness=5)

    #画真实的线圈,
    #cv2.circle(img_tpl, (int(translate_y(8.5)), int(translate_x(72))), 25, (0, 0, 255))

    # 线圈是矩形,用户真实需要的需求
    cv2.putText(img_tpl,"Coil",(int(translate_y(y_pos+3)),int(translate_x(x_pos-1))), cv2.FONT_ITALIC, 0.6, (0, 0, 0), 2)
    cv2.rectangle(img_tpl, (int(translate_y(y_pos)), int(translate_x(x_pos))), (int(translate_y(y_pos-w_)), int(translate_x(x_pos+l_))), (255, 0, 0))

    # 画停止线
    counter=0
    for sl in stop_line if stop_line is not None else []:
        sl = int(translate_x(sl))
        if counter ==0:
            pass
        else: 
            cv2.putText(img_tpl,"Radar StopD",(10, sl), cv2.FONT_ITALIC, 0.6, (0, 0, 0), 2)
        cv2.line(img_tpl, (0, sl), (edge, sl), (0, 0, 255))
        counter+=1
    #画车道线
    for ll in lane_line if lane_line is not None else []:
        ll = int(translate_y(ll))
        cv2.line(img_tpl, (ll, 0), (ll, edge), (0, 0, 255))
    #画测量线,横线判定距离的
    for ml in meas_line:
        _ml = ml
        ml = int(translate_x(ml))
        cv2.line(img_tpl, (0, ml), (edge, ml), (255, 255, 0))
        cv2.putText(img_tpl, '%3d' % _ml, (10, ml), cv2.FONT_ITALIC, 0.4, (0, 0, 0), 1)
    #img_tpl是模板，划线的模板
    img = np.copy(img_tpl)
    remain_deleted = []
    old_em_msg = ''

    i = 0
    cv2.namedWindow("pc0%s"%radar_no,1)
    while ecal_core.ok():
        _, msg, _ = sub408.receive()
        #打印出数字出来
        info = '%d' % enter_coil_count
        cv2.putText(img, info, (int(translate_y(15)), int(translate_x(65))), cv2.FONT_ITALIC, 0.9, (255, 0, 0), 1)

        i += 1
        if i >= 1:
            cv2.imshow('pc0%s' % radar_no, img)
            key = cv2.waitKeyEx(1)
            if key == 27:  # Esc
                ecal_core.finalize()
                cv2.destroyAllWindows()
                exit(0)
            elif key == 43:  # +
                azimuth += 0.01
            elif key == 45:  # -
                azimuth -= 0.01
            elif key == 2490368:  # move up
                azimuth += 0.001
            elif key == 2621440:  # move down
                azimuth -= 0.001
            elif key == 2555904:  # ->(move right)
                img_tpl = np.ones((edge, edge, 3), np.uint8)
                img_tpl *= 255
                for i in range(len(lane_line)):
                    lane_line[i] -= 0.2
                    ll = int(translate_y(lane_line[i]))
                    cv2.line(img_tpl, (ll, 0), (ll, edge), (0, 0, 255))
                    #配车道线使用的，临时的tmp
                    print('%.2f' % lane_line[i], end=',')
                print('')
                pass
            elif key == 2424832:  # <-(move left)
                img_tpl = np.ones((edge, edge, 3), np.uint8)
                img_tpl *= 255
                for i in range(len(lane_line)):
                    lane_line[i] += 0.2
                    ll = int(translate_y(lane_line[i]))
                    cv2.line(img_tpl, (ll, 0), (ll, edge), (0, 0, 255))
                    print('%.2f' % lane_line[i], end=',')
                print('')
            if key != -1:
                print('key:%d' % key)

            i = 0
            img = np.copy(img_tpl)

        count = 0
        frame_ids = set()

        for ele in msg.data:
            _id = ele.obj_id
            x = ele.obj_long_displ_m
            y = ele.obj_lat_displ_m
            vx = ele.obj_vrel_long_ms
            vy = ele.obj_lat_speed_ms
            prop = ele.obj_dyn_prob
            rcs= ele.obj_rcs_value_d_bm2
            measState = ele.obj_meas_stat
            poe =ele.obj_prob_of_exist
            _l = ele.obj_length
            _w =ele.obj_width
            class_type = ele.obj_class
            print(_id,x,y,vx,vy,prop,rcs,measState,poe,_l,_w,class_type)
             #为了展示看的距离设定的
            if float(x) > max_distance:
                continue

            count += 1
            #set，每帧有哪些车辆，每帧会清掉
            frame_ids.add(_id)

            _rcs = rcs
            _x, _y = x, y
            vx, vy = float(vx), float(vy)
            _vx, _vy = vx, vy
            #方位角修正，从雷达收到的原始数据修正
            vx = math.cos(azimuth) * _vx - math.sin(azimuth) * _vy
            vy = math.cos(azimuth) * _vy + math.sin(azimuth) * _vx

            #根据速度计算相对雷达的角度
            orientation = math.atan2(abs(-vy), abs(vx))
            if vx >= 0 and -vy <= 0:
                orientation = -orientation
            elif vx <= 0 and -vy >= 0:
                orientation = math.pi - orientation
            elif vx <= 0 and -vy <= 0:
                orientation = -math.pi + orientation
            orientation = wrap_angle_once(orientation)

            xs, ys = float(x), float(y)
            x = math.cos(azimuth) * xs - math.sin(azimuth) * ys
            y = math.cos(azimuth) * ys + math.sin(azimuth) * xs


            #做非机动车统计，进入线圈，特殊的case
            if _id not in enter_coil_ids:
                if has_enter_coil(x, y):
                    if abs(orientation) < math.pi/4 or abs(abs(orientation) - math.pi) < math.pi/4:
                        enter_coil_ids.add(_id)
                        enter_coil_count += 1
                    else:
                        print('OOO:%s,%d', _id, orientation)

            # x = abs(float((float(x) - max_distance/2) * scale_x) - float(edge/2))
            #折算到pixel 坐标
            x = translate_x(x)
            # y = -float(y) * scale_y + float(edge/2)
            y = translate_y(y)
            if y < 0 or y > edge:
                continue

            x, y = int(x), int(y)
            prop = int(prop)
            rcs = int(rcs)


            #拉长点画图，便于区分颜色
            rcs *= 4
            r = g = b = 0
            if rcs > 0:
                r = rcs
                r = 255 if r > 255 else r
            else:
                b = abs(rcs)
                b = 255 if b > 255 else b
                
            #画识别目标小点
            cv2.circle(img, (y, x), 3, (b, g, r), thickness=1)
            #画尾翼角度。
            arrow(img, orientation, y, x, 18, (b, g, r))

            # info = ' %s-%s,%.2f,%.2f' % (_id, poe, vx, vy)
            info = ' %s' % _id
            cv2.putText(img, info, (y, x-1), cv2.FONT_ITALIC, 0.4, (b, g, r), 1)

            _l = float(_l)*5
            _w = float(_w)*5
            #画十字架，代表车的长宽
            cv2.line(img, (y, int(x-_l/2)), (y, int(x+_l/2)), (b, g, r))
            cv2.line(img, (int(y-_w/2), x), (int(y+_w/2), x), (b, g, r))
        
    #非机动车统计
    for eci in enter_coil_ids.copy():
        if eci not in frame_ids:
            enter_coil_ids.remove(eci)

    cv2.putText(img, '%d' % count, (5, 30), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 0, 255), 1)
    #调整过后的方位角显示
    cv2.putText(img, 'azimuth: %.4f' % azimuth, (5, 50), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 0), 1)
       

if __name__ == "__main__":
    print("begin to show 408 from protobuf!!<<<")
    sys.argv[1]="215"
    radar_no = str(sys.argv[1])
    main(radar_no)
    
    




begin to show 408 from protobuf!!<<<
eCAL v5.9.10 (26.08.2022)

86 27.4000244140625 15.199996948242188 -0.5 1.5 1 5.5 2 7 2 3 6
79 28.79998779296875 18.599990844726562 0.0 0.25 1 7.0 2 7 2 3 6
78 32.0 15.199996948242188 0.0 0.75 1 2.5 3 5 2 3 6
60 31.60003662109375 36.0 0.0 0.5 1 24.0 3 6 2 3 6
33 47.4000244140625 -23.0 0.0 -0.5 1 4.5 3 5 2 3 6
53 54.20001220703125 -4.4000091552734375 13.0 0.25 0 7.0 2 7 2 3 3
92 77.79998779296875 -12.600006103515625 6.25 0.0 0 0.0 3 4 1 1 6
82 92.60003662109375 -6.1999969482421875 7.25 0.25 0 14.5 2 7 4 3 1
52 71.79998779296875 63.600006103515625 0.0 0.25 1 18.5 2 7 2 3 6
57 78.20001220703125 59.0 0.0 0.5 1 19.0 2 7 2 3 6
43 114.60003662109375 -5.4000091552734375 9.0 0.25 0 11.0 2 7 4 3 1
20 119.79998779296875 10.599990844726562 -5.75 -0.25 2 9.5 2 6 4 3 1
87 131.79998779296875 -4.4000091552734375 9.5 0.25 0 11.5 2 7 4 3 1
30 135.0 10.199996948242188 -10.0 -0.25 2 3.0 2 5 1 1 6
75 149.0 -3.600006103515625 8.75 0.25 0 8.5 3 6 1 1 6
59 155.0 10.19999694

47 27.0 -4.8000030517578125 12.75 0.0 0 -6.5 2 3 1 1 6
23 28.60003662109375 -7.4000091552734375 10.75 -0.5 0 -1.0 2 7 2 3 3
86 27.20001220703125 16.199996948242188 -0.25 2.0 1 6.5 2 7 2 3 6
41 27.20001220703125 19.199996948242188 0.0 0.25 1 8.5 2 7 2 3 6
79 28.79998779296875 18.599990844726562 0.0 0.25 1 6.5 3 5 2 3 6
60 31.60003662109375 36.19999694824219 0.0 0.5 1 24.5 3 5 2 3 6
24 41.0 -27.600006103515625 -1.25 0.25 1 3.0 2 5 2 3 6
33 47.4000244140625 -23.199996948242188 0.0 -0.5 1 4.0 3 6 2 3 6
53 59.79998779296875 -4.4000091552734375 13.25 0.25 0 6.0 2 7 2 3 3
92 80.4000244140625 -12.800003051757812 6.0 0.0 0 2.0 3 6 1 1 6
82 95.79998779296875 -6.1999969482421875 7.5 0.25 0 11.0 2 7 4 3 1
20 117.4000244140625 10.800003051757812 -5.75 0.0 2 10.0 2 7 4 3 1
43 118.60003662109375 -5.0 9.25 0.5 0 12.5 2 7 4 3 1
30 130.79998779296875 10.0 -10.0 -0.25 2 3.0 3 3 1 1 6
87 136.0 -4.1999969482421875 9.5 0.25 0 10.5 2 7 4 3 1
59 151.20001220703125 10.0 -8.75 -0.25 2 4.5 3 5 4 3 1
75 152.79998

89 25.79998779296875 17.599990844726562 0.25 1.25 1 7.5 2 7 2 3 6
47 35.60003662109375 -4.600006103515625 13.25 0.25 0 1.0 2 7 1 1 6
23 35.79998779296875 -6.8000030517578125 10.75 0.5 0 -2.5 2 6 2 3 3
30 48.0 7.0 0.0 1.0 1 6.0 3 2 2 3 6
21 48.20001220703125 -21.600006103515625 0.0 -0.75 1 5.5 3 5 2 3 6
53 68.60003662109375 -4.1999969482421875 13.5 0.25 0 5.5 2 6 2 3 3
57 78.20001220703125 59.399993896484375 0.0 0.5 1 17.5 2 7 2 3 6
82 101.0 -6.0 8.0 0.25 0 14.0 2 7 4 3 1
20 113.60003662109375 10.800003051757812 -5.75 0.0 2 7.5 2 7 4 3 1
43 124.60003662109375 -4.8000030517578125 9.5 0.5 0 10.5 2 7 4 3 1
87 142.0 -4.4000091552734375 9.0 0.0 0 9.5 2 7 4 3 1
59 145.60003662109375 9.800003051757812 -8.75 -0.25 2 4.5 3 2 4 3 1
75 158.4000244140625 -3.4000091552734375 8.25 0.25 0 6.5 2 7 1 1 6
83 169.79998779296875 -3.4000091552734375 7.0 0.25 0 8.0 2 7 4 3 1
22 184.4000244140625 0.399993896484375 7.75 0.5 0 19.5 2 7 1 1 6
44 186.0 11.599990844726562 -8.5 -0.25 2 8.5 2 7 4 3 1
8 186.600036621

59 28.0 -7.4000091552734375 10.5 0.0 0 0.5 1 3 1 1 6
50 24.4000244140625 20.399993896484375 1.25 0.0 0 5.0 1 3 1 1 6
89 26.0 19.199996948242188 1.0 2.25 4 2.5 3 5 2 3 6
23 43.0 -6.8000030517578125 11.0 0.25 0 6.0 2 7 2 3 3
47 44.4000244140625 -4.4000091552734375 13.75 0.25 0 8.5 2 7 1 1 6
30 47.79998779296875 -3.0 0.25 -0.75 1 7.0 2 7 2 3 6
33 46.79998779296875 -23.400009155273438 0.0 -0.25 1 7.0 3 6 2 3 6
39 44.60003662109375 -28.800003051757812 -1.25 0.0 0 0.5 3 3 1 1 6
67 56.60003662109375 41.19999694824219 0.0 0.25 1 10.0 2 7 2 3 6
2 69.4000244140625 13.0 0.0 1.25 1 5.0 3 6 2 3 6
53 77.20001220703125 -4.0 13.25 0.25 0 12.5 2 7 2 3 3
21 68.0 43.59999084472656 0.0 0.5 1 8.5 3 3 2 3 6
57 78.0 59.79998779296875 0.0 0.5 1 18.0 2 7 2 3 6
82 106.20001220703125 -6.0 8.5 0.25 0 7.5 2 7 4 3 1
20 110.0 11.0 -6.0 0.0 2 5.0 2 6 4 3 1
43 130.60003662109375 -4.600006103515625 9.0 0.25 0 11.0 2 7 4 3 1
92 139.60003662109375 9.599990844726562 -8.75 0.0 2 3.0 3 3 1 1 6
87 147.79998779296875 -4.19999

50 24.60003662109375 21.0 1.5 1.5 0 3.5 2 7 2 3 3
59 32.60003662109375 -7.600006103515625 10.75 0.25 0 1.0 2 7 2 3 3
81 39.0 -22.199996948242188 -0.75 0.75 1 0.5 3 4 2 3 6
23 47.79998779296875 -6.8000030517578125 11.25 0.25 0 5.0 2 7 2 3 3
47 50.4000244140625 -4.1999969482421875 14.0 0.25 0 9.5 2 7 1 1 6
96 50.4000244140625 17.599990844726562 0.0 0.75 1 4.0 3 6 2 3 6
53 82.79998779296875 -4.0 13.25 0.25 0 14.0 2 7 2 3 3
57 77.79998779296875 60.20001220703125 0.0 0.75 1 18.0 2 7 2 3 6
94 84.20001220703125 60.79998779296875 0.0 0.25 1 16.5 2 7 2 3 6
20 107.4000244140625 10.800003051757812 -6.0 0.0 2 7.0 2 7 4 3 1
82 110.0 -5.8000030517578125 8.75 0.25 0 10.5 2 7 4 3 1
43 134.4000244140625 -4.600006103515625 9.0 0.25 0 10.5 2 7 4 3 1
87 151.60003662109375 -4.0 8.5 0.25 0 14.5 2 7 4 3 1
75 167.20001220703125 -3.1999969482421875 7.75 0.25 0 7.5 2 5 4 3 1
44 176.79998779296875 11.199996948242188 -8.5 -0.25 2 6.5 2 7 4 3 1
83 177.20001220703125 -3.0 6.5 0.25 0 10.0 2 7 4 3 1
8 191.79998779296

...

50 21.4000244140625 28.800003051757812 0.0 5.0 4 6.0 3 6 4 4 3
82 42.0 -7.1999969482421875 2.0 0.25 0 4.5 2 7 1 1 6
39 39.20001220703125 -25.0 0.0 -0.5 1 7.5 2 6 2 3 6
59 46.0 -25.199996948242188 0.0 -0.25 1 7.0 3 5 2 3 6
43 53.60003662109375 -7.0 2.25 0.0 0 7.5 2 7 2 3 3
17 50.20001220703125 -32.40000915527344 0.0 -0.25 1 10.0 2 6 2 3 6
46 50.60003662109375 36.59999084472656 -5.25 0.5 2 4.5 3 5 1 1 6
87 66.4000244140625 -6.600006103515625 3.75 0.25 0 5.5 2 7 2 3 3
92 72.4000244140625 5.399993896484375 -1.0 0.0 2 5.0 2 7 4 3 1
75 77.20001220703125 -6.600006103515625 4.5 0.0 0 7.5 2 7 1 1 6
97 66.4000244140625 40.59999084472656 0.0 2.25 1 11.0 2 7 2 3 6
67 83.79998779296875 8.800003051757812 -8.75 -0.25 2 0.5 2 6 1 1 6
83 88.0 -6.0 4.5 0.0 0 17.5 2 7 4 3 1
15 71.79998779296875 63.600006103515625 0.0 0.25 1 19.5 2 7 2 3 6
22 100.60003662109375 -5.4000091552734375 5.5 0.25 0 14.0 2 7 1 1 6
45 102.79998779296875 -3.4000091552734375 12.75 0.25 0 6.5 2 7 4 3 1
81 109.0 6.8000030517578125 -2.

79 27.20001220703125 19.0 0.0 0.25 1 14.5 2 7 3 4 1
50 21.4000244140625 31.0 0.0 5.0 4 6.0 3 5 4 4 3
82 43.0 -7.4000091552734375 2.5 0.0 0 1.0 2 7 1 1 6
39 39.20001220703125 -25.199996948242188 0.0 -0.5 1 7.5 3 5 2 3 6
59 45.79998779296875 -25.199996948242188 0.0 -0.25 1 8.0 3 6 2 3 6
43 54.60003662109375 -6.8000030517578125 2.75 0.25 0 11.0 2 7 2 3 3
38 47.20001220703125 -32.80000305175781 0.0 -0.25 1 20.5 2 7 2 3 6
46 48.4000244140625 36.19999694824219 -4.75 0.0 2 4.5 3 5 1 1 6
87 68.20001220703125 -6.8000030517578125 4.0 0.0 0 6.5 2 7 2 3 3
92 72.0 5.399993896484375 -1.0 -0.25 2 2.5 2 7 4 3 1
97 66.60003662109375 41.0 0.0 1.5 1 14.5 2 7 2 3 6
75 79.0 -6.600006103515625 4.25 0.0 0 7.5 2 6 1 1 6
67 80.20001220703125 8.800003051757812 -8.5 -0.25 2 -2.0 3 3 1 1 6
83 90.0 -6.0 5.0 0.25 0 9.5 2 7 4 3 1
15 71.79998779296875 63.79998779296875 0.0 0.25 1 19.5 3 5 2 3 6
22 103.0 -5.4000091552734375 5.5 0.25 0 17.0 2 7 1 1 6
81 107.79998779296875 6.5999908447265625 -2.25 -0.25 2 3.0 3 2 4 3 1


27 25.0 26.599990844726562 0.0 0.25 1 12.5 2 7 2 3 6
50 21.4000244140625 33.0 0.0 5.0 4 6.0 3 2 2 3 3
82 44.0 -7.4000091552734375 2.5 0.0 0 2.5 2 7 1 1 6
39 39.4000244140625 -25.400009155273438 0.0 -0.5 1 7.0 3 5 2 3 6
59 45.79998779296875 -25.600006103515625 0.0 -0.5 1 8.5 2 7 2 3 6
81 44.60003662109375 -30.199996948242188 1.5 0.0 0 0.0 3 3 1 1 6
43 56.0 -6.8000030517578125 3.25 0.25 0 12.0 2 7 2 3 3
41 33.20001220703125 47.399993896484375 0.0 0.25 1 31.5 3 4 2 3 6
46 46.4000244140625 36.80000305175781 -4.25 0.25 2 4.5 3 6 1 1 6
87 70.0 -6.600006103515625 4.5 0.0 0 7.0 2 7 2 3 3
97 66.60003662109375 41.19999694824219 0.0 1.0 1 14.5 2 7 2 3 6
75 80.79998779296875 -6.4000091552734375 4.5 0.0 0 9.0 2 7 1 1 6
83 92.20001220703125 -6.0 5.5 0.0 0 14.0 2 7 4 3 1
22 105.4000244140625 -5.1999969482421875 5.75 0.25 0 18.0 2 7 1 1 6
45 113.60003662109375 -2.8000030517578125 12.5 0.5 0 9.0 2 7 4 3 1
1 119.79998779296875 -4.600006103515625 7.0 0.5 0 7.0 2 6 4 3 1
8 135.4000244140625 -13.1999969482

82 45.0 -7.4000091552734375 2.5 0.0 0 4.0 2 7 1 1 6
65 40.79998779296875 -27.0 0.0 -0.25 1 8.0 3 6 2 3 6
81 43.4000244140625 -24.199996948242188 0.0 -0.25 1 6.0 2 6 2 3 6
59 45.79998779296875 -26.0 0.0 -0.5 1 7.5 2 6 2 3 6
43 57.60003662109375 -6.8000030517578125 3.75 0.0 0 10.0 2 7 2 3 3
46 44.60003662109375 37.19999694824219 -4.0 0.5 2 5.0 3 5 1 1 6
87 71.79998779296875 -6.4000091552734375 4.75 0.25 0 5.0 2 7 2 3 3
97 66.60003662109375 41.19999694824219 0.0 0.75 1 14.5 2 7 2 3 6
75 83.0 -6.1999969482421875 5.0 0.25 0 5.0 2 6 1 1 6
83 94.60003662109375 -6.0 5.75 0.0 0 15.0 2 7 4 3 1
15 71.79998779296875 64.0 0.0 0.25 1 19.0 3 5 2 3 6
22 107.79998779296875 -5.1999969482421875 5.75 0.25 0 18.5 2 7 1 1 6
45 119.20001220703125 -2.4000091552734375 13.0 0.5 0 9.0 2 7 4 3 1
1 122.79998779296875 -4.600006103515625 7.25 0.5 0 9.5 2 7 4 3 1
8 136.0 -13.199996948242188 1.5 0.0 0 17.0 2 7 4 3 1
36 139.60003662109375 -10.800003051757812 4.0 0.25 0 12.5 2 7 4 3 1
13 145.79998779296875 -4.4000091552

KeyboardInterrupt: 